# Dataset 教学
url：https://zhuanlan.zhihu.com/p/30751039

In [7]:
import tensorflow as tf
import numpy as np
import tensorflow.contrib.eager as tfe
#tfe.enable_eager_execution()

In [2]:
dataset = tf.data.Dataset.from_tensor_slices(np.array([1.0, 2.0, 3.0, 4.0, 5.0]))


## 读取DataSet方法

In [3]:
iterator = dataset.make_one_shot_iterator()
one_element = iterator.get_next()
with tf.Session() as sess:
    for i in range(5):
        print (sess.run(one_element))
    sess.close()

1.0
2.0
3.0
4.0
5.0


## 异常处理
如果一个dataset中元素被读取完了，再尝试sess.run(one_element)的话，就会抛出tf.errors.OutOfRangeError异常，这个行为与使用队列方式读取数据的行为是一致的。在实际程序中，可以在外界捕捉这个异常以判断数据是否读取完，请参考下面的代码：

In [4]:
dataset = tf.data.Dataset.from_tensor_slices(np.array([1.0, 2.0, 3.0, 4.0, 5.0]))
iterator = dataset.make_one_shot_iterator()
one_element = iterator.get_next()
with tf.Session() as sess:
    try:
        while True:
            print (sess.run(one_element))
    except:
        print ("Read completed")

1.0
2.0
3.0
4.0
5.0
Read completed


## Eager 模式
不需要sess.run()，而是直接迭代取值

In [5]:

#dataset = tf.data.DataSet.from_tensor_slices(np.array([1.0, 2.0, 3.0, 4.0, 5.0]))
#for one_element in tfe.Iterator(dataset):
#    print (one_element)
    

## 创建更复杂的Dataset
注意输出，函数会分别切分"a"中的数值以及"b"中的数值，最终dataset中的一个元素就是类似于{"a": 1.0, "b": [0.9, 0.1]}的形式

In [13]:
dataset = tf.data.Dataset.from_tensor_slices(
    {
        "a": np.array([1.0, 2.0, 3.0, 4.0, 5.0]),
        "b": np.random.uniform(size=(5, 2)),
        "x": np.array([[1.0, 2.0], [2.0, 3.0], [4.0, 3.0], [3.0, 2.0], [3.1, 2.4]])
    })
iterator = dataset.make_one_shot_iterator()
one_element = iterator.get_next()
with tf.Session() as sess:
    for i in range(5):
        print (sess.run(one_element))
    sess.close()

{'a': 1.0, 'b': array([ 0.39050292,  0.82127424]), 'x': array([ 1.,  2.])}
{'a': 2.0, 'b': array([ 0.75659616,  0.89930735]), 'x': array([ 2.,  3.])}
{'a': 3.0, 'b': array([ 0.28180304,  0.85748123]), 'x': array([ 4.,  3.])}
{'a': 4.0, 'b': array([ 0.50802494,  0.42912675]), 'x': array([ 3.,  2.])}
{'a': 5.0, 'b': array([ 0.32033154,  0.53173986]), 'x': array([ 3.1,  2.4])}


{'a': 1.0, 'b': array([ 0.39050292,  0.82127424]), 'x': array([ 1.,  2.])}
{'a': 2.0, 'b': array([ 0.75659616,  0.89930735]), 'x': array([ 2.,  3.])}
{'a': 3.0, 'b': array([ 0.28180304,  0.85748123]), 'x': array([ 4.,  3.])}
{'a': 4.0, 'b': array([ 0.50802494,  0.42912675]), 'x': array([ 3.,  2.])}
{'a': 5.0, 'b': array([ 0.32033154,  0.53173986]), 'x': array([ 3.1,  2.4])}

## Transformation
对Dataset中的元素做变换：Transformation
Dataset支持一类特殊的操作：Transformation。一个Dataset通过Transformation变成一个新的Dataset。通常我们可以通过Transformation完成数据变换，打乱，组成batch，生成epoch等一系列操作。

#### 常用的Transformation有：

map
batch
shuffle
repeat
下面就分别进行介绍。

#### （1）map

map接收一个函数，Dataset中的每个元素都会被当作这个函数的输入，并将函数返回值作为新的Dataset，如我们可以对dataset中每个元素的值加1：

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(np.array([1.0, 2.0, 3.0, 4.0, 5.0]))
dataset = dataset.map(lambda x: x + 1) # 2.0, 3.0, 4.0, 5.0, 6.0
iterator = dataset.make_one_shot_iterator()
one_element = iterator.get_next()
with tf.Session() as sess:
    for i in range(5):
        print (sess.run(one_element))
    sess.close()

In [22]:
dataset = tf.data.Dataset.from_tensor_slices(
    {
        "a": np.array([1.0, 2.0, 3.0, 4.0, 5.0]),
        "b": np.random.uniform(size=(5, 2)),
        "x": np.array([[1.0, 2.0], [2.0, 3.0], [4.0, 3.0], [3.0, 2.0], [3.1, 2.4]])
    })
dataset = dataset.map(lambda x: (x["a"] + 1, x["b"], x["x"]))
iterator = dataset.make_one_shot_iterator()
one_element = iterator.get_next()
with tf.Session() as sess:
    for i in range(5):
        print (sess.run(one_element))
    sess.close()

(2.0, array([ 0.47752821,  0.49427107]), array([ 1.,  2.]))
(3.0, array([ 0.41259332,  0.41796848]), array([ 2.,  3.]))
(4.0, array([ 0.15880128,  0.91545947]), array([ 4.,  3.]))
(5.0, array([ 0.81820705,  0.21815505]), array([ 3.,  2.]))
(6.0, array([ 0.10130723,  0.83601855]), array([ 3.1,  2.4]))


#### batch

dataset = dataset.batch(2)
iterator = dataset.make_one_shot_iterator()
one_element = iterator.get_next()
with tf.Session() as sess:
    for i in range(2):
        print (sess.run(one_element))
    sess.close()

#### shuffle

dataset = dataset.shuffle(buffer_size=3)
iterator = dataset.make_one_shot_iterator()
one_element = iterator.get_next()
with tf.Session() as sess:
    for i in range(5):
        print (sess.run(one_element))
    sess.close()

dataset = dataset.repeat(5)
iterator = dataset.make_one_shot_iterator()
one_element = iterator.get_next()
with tf.Session() as sess:
    for i in range(20):
        print (sess.run(one_element))
    sess.close()

#### 例子：读入磁盘图片与对应label
讲到这里，我们可以来考虑一个简单，但同时也非常常用的例子：读入磁盘中的图片和图片相应的label，并将其打乱，组成batch_size=32的训练样本。在训练时重复10个epoch。

对应的程序为（从官方示例程序修改而来）：

###### 函数的功能时将filename对应的图片文件读进来，并缩放到统一的大小
def _parse_function(filename, label):
  image_string = tf.read_file(filename)
  image_decoded = tf.image.decode_image(image_string)
  image_resized = tf.image.resize_images(image_decoded, [28, 28])
  return image_resized, label

###### 图片文件的列表
filenames = tf.constant(["/var/data/image1.jpg", "/var/data/image2.jpg", ...])
###### label[i]就是图片filenames[i]的label
labels = tf.constant([0, 37, ...])

###### 此时dataset中的一个元素是(filename, label)
dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))

###### 此时dataset中的一个元素是(image_resized, label)
dataset = dataset.map(_parse_function)

###### 此时dataset中的一个元素是(image_resized_batch, label_batch)
dataset = dataset.shuffle(buffersize=1000).batch(32).repeat(10)

在这个过程中，dataset经历三次转变：

运行dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))后，dataset的一个元素是(filename, label)。filename是图片的文件名，label是图片对应的标签。
之后通过map，将filename对应的图片读入，并缩放为28x28的大小。此时dataset中的一个元素是(image_resized, label)
最后，dataset.shuffle(buffersize=1000).batch(32).repeat(10)的功能是：在每个epoch内将图片打乱组成大小为32的batch，并重复10次。最终，dataset中的一个元素是(image_resized_batch, label_batch)，image_resized_batch的形状为(32, 28, 28, 3)，而label_batch的形状为(32, )，接下来我们就可以用这两个Tensor来建立模型了。
Dataset的其它创建方法....
除了tf.data.Dataset.from_tensor_slices外，目前Dataset API还提供了另外三种创建Dataset的方式：

tf.data.TextLineDataset()：这个函数的输入是一个文件的列表，输出是一个dataset。dataset中的每一个元素就对应了文件中的一行。可以使用这个函数来读入CSV文件。
tf.data.FixedLengthRecordDataset()：这个函数的输入是一个文件的列表和一个record_bytes，之后dataset的每一个元素就是文件中固定字节数record_bytes的内容。通常用来读取以二进制形式保存的文件，如CIFAR10数据集就是这种形式。
tf.data.TFRecordDataset()：顾名思义，这个函数是用来读TFRecord文件的，dataset中的每一个元素就是一个TFExample。
它们的详细使用方法可以参阅文档：Module: tf.data

更多类型的Iterator....
在非Eager模式下，最简单的创建Iterator的方法就是通过dataset.make_one_shot_iterator()来创建一个one shot iterator。除了这种one shot iterator外，还有三个更复杂的Iterator，即：

initializable iterator
reinitializable iterator
feedable iterator
initializable iterator必须要在使用前通过sess.run()来初始化。使用initializable iterator，可以将placeholder代入Iterator中，这可以方便我们通过参数快速定义新的Iterator。一个简单的initializable iterator使用示例：

limit = tf.placeholder(dtype=tf.int32, shape=[])

dataset = tf.data.Dataset.from_tensor_slices(tf.range(start=0, limit=limit))

iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

with tf.Session() as sess:
    sess.run(iterator.initializer, feed_dict={limit: 10})
    for i in range(10):
      value = sess.run(next_element)
      assert i == value
此时的limit相当于一个“参数”，它规定了Dataset中数的“上限”。

initializable iterator还有一个功能：读入较大的数组。

在使用tf.data.Dataset.from_tensor_slices(array)时，实际上发生的事情是将array作为一个tf.constants保存到了计算图中。当array很大时，会导致计算图变得很大，给传输、保存带来不便。这时，我们可以用一个placeholder取代这里的array，并使用initializable iterator，只在需要时将array传进去，这样就可以避免把大数组保存在图里，示例代码为（来自官方例程）：

# 从硬盘中读入两个Numpy数组
with np.load("/var/data/training_data.npy") as data:
  features = data["features"]
  labels = data["labels"]

features_placeholder = tf.placeholder(features.dtype, features.shape)
labels_placeholder = tf.placeholder(labels.dtype, labels.shape)

dataset = tf.data.Dataset.from_tensor_slices((features_placeholder, labels_placeholder))
iterator = dataset.make_initializable_iterator()
sess.run(iterator.initializer, feed_dict={features_placeholder: features,
                                          labels_placeholder: labels})
reinitializable iterator和feedable iterator相比initializable iterator更复杂，也更加少用，如果想要了解它们的功能，可以参阅官方介绍，这里就不再赘述了。

总结
本文主要介绍了Dataset API的基本架构：Dataset类和Iterator类，以及它们的基础使用方法。

在非Eager模式下，Dataset中读出的一个元素一般对应一个batch的Tensor，我们可以使用这个Tensor在计算图中构建模型。

在Eager模式下，Dataset建立Iterator的方式有所不同，此时通过读出的数据就是含有值的Tensor，方便调试。

作为兼容两种模式的Dataset API，在今后应该会成为TensorFlow读取数据的主流方式。关于Dataset API的进一步介绍，可以参阅下面的资料：

Importing Data ：官方Guide
Module: tf.data： API文档
Introduction to TensorFlow Datasets and Estimators：如何联合使用Dataset和Estimator